Imports (**Note: install `ipywidgets` and `nodejs` in environment if they are not already**)

OM 501<br>
Vehicle Routing Project Guidelines<br>
DUE DATE: December 4 (wednesday), 2022 @ 6:30pm - 8:30pm.<br>

Description:
The purpose of this team project is to design, construct, and use a Python application that is capable of solving a capacitated vehicle routing problem with multiple demand points and suggesting a distribution strategy for a subset of U.S. states. For the purpose of this project, a distribution strategy specifies: 
- how many distribution centers (DCs) to open, 
- where to open them, 
- the customer assigned to each DC,
- and data insights to support your recommendation.

The data you will be using is based on Home Depot locations in the states of California (CA), Texas (TX), and Alabama (AL). You will be determining solutions for each state.
Problem assumptions are: 
- Home Depot distributes goods to "customer" stores using a single DC and makes weekly shipments.
- Home Depot considers each state separately, i.e., DCs in AL ship only to stores in AL, 
- Demand is given in pounds. Essentially, we are assuming that the products are "heavy", thus, when shipping, we are more constrained by weight rather than volume.
- You have access to vehicles that can haul 40,000 in each trip.

Presentations:
Each team will give a 20 minute presentation detailing their approach to the problem, along with their results for each state and any justification.

In [17]:
import dateutil
import itertools
import json
import pathlib

import bs4
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
sns.set_style('whitegrid')


from gurobipy import *
from ipywidgets import interact

import datetime
from warnings import filterwarnings
filterwarnings('ignore')

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

## Distance Function

In [18]:
def get_distance_data(df):

    def haversine_np(lon1, lat1, lon2, lat2):
        """
        Calculate the great circle distance between two points
        on the earth (specified in decimal degrees)

        All args must be of equal length.    

        """
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

        c = 2 * np.arcsin(np.sqrt(a))
        miles = 6367 * c * 0.621371
        return miles


    store_ids = df['store_id'].unique().tolist()
    store_ids_product = list(itertools.product(store_ids, store_ids))
    dm = pd.DataFrame(store_ids_product, columns=['from', 'to'])
    lat_mapper = df.set_index('store_id')['lat'].to_dict()
    lon_mapper = df.set_index('store_id')['lon'].to_dict()
    dm['from_lat'] = dm['from'].map(lat_mapper)
    dm['from_lon'] = dm['from'].map(lon_mapper)
    dm['to_lat'] = dm['to'].map(lat_mapper)
    dm['to_lon'] = dm['to'].map(lon_mapper)
    
    lon1, lat1, lon2, lat2 = map(
        np.radians, 
        [dm['from_lon'], 
        dm['from_lat'], 
        dm['to_lon'], 
        dm['to_lat'],
        ]
    )

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    dm['miles'] = 6367 * c * 0.621371
    
    dm = dm.pivot(
        index='from',
        columns='to',
        values='miles'
    )
    
    return dm

## Store Data

In [19]:
data_dir = pathlib.Path('data')
data_dir.mkdir(exist_ok=True)

store_data_filepath = pathlib.Path(data_dir, 'stores.csv')

if store_data_filepath.exists():
    store_data = pd.read_csv(store_data_filepath)

store_data.head()

,store_name,state,city,zip,store_id,lat,lon
0,Manchester-Road,MO,Ballwin,63011,3004,38.6091,-90.5598
1,Ellisville,MO,Ellisville,63011,3018,38.6091,-90.5598
2,West-Frederick,MD,Frederick,21702,2511,39.4926,-77.4612
3,I-10-Bullard,LA,New-Orleans,70128,352,30.0487,-89.9585
4,Crystal-River,FL,Crystal-River,34429,6332,28.8547,-82.6669


## Get Data for a Single State and Demand Data

In [20]:
state = 'AL'

state_df = store_data[store_data['state']==state]
state_df = state_df.reset_index(drop=True)
coordinates = state_df.set_index('store_id')[['lat', 'lon']].copy()

distance_matrix = get_distance_data(state_df)

demand_data_filepath = pathlib.Path(data_dir, 'demand_data.json')
with open(demand_data_filepath) as fin:
    demand_dict = json.load(fin)

## Demand Data for Selected State

In [22]:
relevant_stores = state_df['store_id'].tolist()
relevant_demand = {key: val for key, val in demand_dict.items() if int(key) in relevant_stores}
relevant_demand_df = pd.DataFrame().from_dict(relevant_demand, orient='columns')
relevant_demand_df.index = pd.to_datetime(relevant_demand_df.index)
relevant_demand_df.columns = [int(val) for val in relevant_demand_df.columns]

In [24]:
relevant_stores

[875,
 882,
 883,
 813,
 805,
 802,
 8577,
 806,
 801,
 817,
 816,
 888,
 6889,
 884,
 810,
 812,
 880,
 809,
 808,
 818,
 866,
 803,
 887,
 863,
 881,
 865,
 877,
 804]

In [25]:
store_data = pd.DataFrame().from_dict(relevant_demand, orient='columns').reset_index()
store_data['Date'] = pd.to_datetime(store_data['index'],  format='%Y-%m-%d')
store_data = store_data.drop(columns=['index'])
store_data = store_data.set_index('Date')
store_data

,875,882,883,813,805,802,8577,806,801,817,...,808,818,866,803,887,863,881,865,877,804
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-05,293,290,318,303,403,468,401,477,486,430,...,512,278,412,349,396,305,463,483,462,265
2020-01-12,297,284,317,316,403,468,400,482,483,427,...,510,284,411,345,402,297,477,492,469,262
2020-01-19,295,286,314,314,406,461,410,485,485,429,...,506,276,414,352,399,302,476,492,471,261
2020-01-26,290,283,315,307,408,480,410,485,491,421,...,513,278,414,350,397,309,465,496,466,259
2020-02-02,285,282,312,304,407,467,416,486,488,423,...,501,283,418,350,392,314,470,491,462,259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-30,191,138,153,410,547,648,490,311,494,300,...,475,415,586,398,332,432,390,614,449,66
2022-02-06,191,136,152,411,543,649,482,315,494,303,...,467,411,588,397,337,435,386,616,457,63
2022-02-13,201,139,151,416,546,654,473,307,493,301,...,476,414,588,391,335,420,397,618,454,64


In [8]:
rmse_list = []

for store in relevant_stores:
    train = pd.DataFrame()
    test = pd.DataFrame()
    

    train = store_data[str(store)][store_data.index <= pd.to_datetime("2021-07-04", format='%Y-%m-%d')]
    test = store_data[str(store)][store_data.index > pd.to_datetime("2021-07-04", format='%Y-%m-%d')]

#     plt.plot(train, color = "black")
#     plt.plot(test, color = "red")
#     plt.ylabel('Alabama Demand')
#     plt.xlabel('Date')
#     plt.xticks(rotation=45)
#     plt.title("Train/Test split for Alabama State Demand")

    from statsmodels.tsa.arima.model import ARIMA
    y = train
    test_dates = list(test.index)
    test_dates.append(pd.to_datetime("2022-06-03", format='%Y-%m-%d'))

    ARIMAmodel = ARIMA(y, order = (2, 2, 2))
    ARIMAmodel = ARIMAmodel.fit()

    y_pred = ARIMAmodel.get_forecast(len(test.index))
    y_pred_df = y_pred.conf_int(alpha = 0.05) 
    y_pred_df["Predictions"] = ARIMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
    y_pred_df.index = test.index
    y_pred_out = y_pred_df["Predictions"] 
#     plt.plot(y_pred_out, color='Green', label = 'ARIMA Predictions')
#     plt.legend()


    arma_rmse = np.sqrt(mean_squared_error(test.values, y_pred_df["Predictions"]))
    rmse_list.append(arma_rmse)
#     print(f'Store number: {store}')
#     print("RMSE: ", arma_rmse)

rmse_list

C:\Users\rider\anaconda3\envs\opt_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  warnings.warn('No frequency information was'
C:\Users\rider\anaconda3\envs\opt_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  warnings.warn('No frequency information was'
C:\Users\rider\anaconda3\envs\opt_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  warnings.warn('No frequency information was'
C:\Users\rider\anaconda3\envs\opt_env\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\rider\anaconda3\envs\opt_env\lib\site-pack

[5.426290440752318,
 2.1218695682383855,
 1.4502247313659657,
 5.611441093692198,
 2.0868481361848494,
 3.9286169215016447,
 8.726954546630319,
 4.217987460523575,
 2.137553733306061,
 5.112524678651804,
 9.144242894480602,
 2.0911880782598726,
 4.611528419846311,
 13.146373066030645,
 1.056294573876079,
 5.119726366241629,
 2.1254827721084046,
 5.881316256262291,
 4.886627901288609,
 4.944300310609846,
 2.257351825952268,
 3.263917723055257,
 7.105527235481937,
 4.201130343855001,
 5.873964595330445,
 3.1138694234806317,
 6.872077468935345,
 1.3140176047666157]

In [26]:
def demand_prediction(customer_number):
    train = pd.DataFrame()
    test = pd.DataFrame()

    train = store_data[str(customer_number)][store_data.index <= pd.to_datetime("2021-07-04", format='%Y-%m-%d')]
    test = store_data[str(customer_number)][store_data.index > pd.to_datetime("2021-07-04", format='%Y-%m-%d')]

    from statsmodels.tsa.arima.model import ARIMA
    y = train
    test_dates = list(test.index)
    test_dates.append(pd.to_datetime("2022-03-06", format='%Y-%m-%d'))

    ARIMAmodel = ARIMA(y, order = (2, 2, 2))
    ARIMAmodel = ARIMAmodel.fit()

    y_pred = ARIMAmodel.get_forecast(len(test_dates))
    y_pred_df = y_pred.conf_int(alpha = 0.05) 
    y_pred_df["Predictions"] = ARIMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
    y_pred_df.index = test_dates
    y_pred_out = y_pred_df["Predictions"] 


    ## Upper 95% Confidence Demand Predicition 
    return y_pred_df.loc['2022-03-06'][1]

In [27]:
store_demand = {}
for store in relevant_stores:
    store_demand[store] = demand_prediction(store)

store_demand

{875: 199.10191305120193,
 882: 134.88837801128835,
 883: 150.0967783104618,
 813: 435.43403752618383,
 805: 553.75756886943,
 802: 670.0922036365653,
 8577: 514.9288177075169,
 806: 312.39378347206343,
 801: 496.96678373879513,
 817: 304.35048218331804,
 816: 624.3757982269314,
 888: 127.10404744736721,
 6889: 207.00416555878766,
 884: 486.90094350777986,
 810: 551.078506637042,
 812: 329.589922030078,
 880: 273.03814791207583,
 809: 337.7330836044712,
 808: 483.2701040560449,
 818: 428.6571353567031,
 866: 603.7856284187529,
 803: 403.5885373755818,
 887: 353.2210666049812,
 863: 449.03769346787357,
 881: 453.2337468457781,
 865: 626.0158715240508,
 877: 457.87455214262303,
 804: 61.988803406850664}

In [28]:
d = distance_matrix.values.tolist()

distance_dict = {}
    
col_num = 0
row_num = 0
tot_num = 0
for col in distance_matrix.columns:
    row_num = 0
    for row in distance_matrix:
        distance_dict[col, row] = d[col_num][row_num]
        row_num += 1
    col_num += 1

In [29]:
## D IS THS DISTANCE MATRIX FOR ONE POINT TO ANOTHER AS A (Origin, Destination) PAIR IN A DICTIONARY
d = distance_dict

In [14]:
vehicle_capacity = 40000
miles_limit = 400

In [53]:
customer_locations = coordinates.index.tolist()

depot_cost_dict = {} 
for test_depot in customer_locations:
    customer_locations.remove(test_depot)

    pairs = []
    for start_customer in customer_locations:
        for end_customer in customer_locations:
            if start_customer < end_customer:
                pairs.append((start_customer, end_customer))

    savings = {}
    for loc1, loc2 in pairs:
        savings[loc1, loc2] = d[test_depot, loc1] + d[test_depot, loc2] - d[loc1, loc2]

    savings_df = pd.DataFrame().from_dict(savings, orient='index')
    savings_df.columns = ['savings']
    savings_df = savings_df.sort_values('savings', ascending=False)
    savings = savings_df['savings'].to_dict()

    trips = {}
    trip_counter = 0
    trip_assignments = {customer: -1 for customer in customer_locations}

    for loc1, loc2 in savings:

        ## IF NEITHER LOCATION IS IN A TRIP
        if (trip_assignments[loc1] == -1) and (trip_assignments[loc2] == -1):
            capacity_required = store_demand[loc1] + store_demand[loc2]

            ## CHECKING THE ORDER TO THE UNCHOSEN POINTS INTO THE TRIP 
            if d[test_depot, loc1] + d[loc1, loc2] < d[test_depot, loc2] + d[loc2, loc1]:
                miles_driven = d[test_depot, loc1] + d[loc1, loc2] + d[loc2, test_depot]
                return_trip = d[loc2, test_depot]

            else:
                miles_driven = d[test_depot, loc2] + d[loc2, loc1] + d[loc1, test_depot]        
                return_trip = d[loc1, test_depot]

            ##CHECKING IF CAPACITY AND MILEAGE ARE WITHIN CONSTRAINTS 
            if capacity_required <= vehicle_capacity and miles_driven <= miles_limit:
                trip_counter += 1
                trips[trip_counter] = {
                    'customers': [loc1, loc2],
                    'capacity_allocated': capacity_required,
                    'miles': miles_driven,
                    'return_trip': return_trip
                }
                ## MAPPING STORES TO TRIP
                trip_assignments[loc1] = trip_counter
                trip_assignments[loc2] = trip_counter

        ## IF FIRST IS IN A TRIP AND SECOND ISN'T
        elif (trip_assignments[loc1] >= 0) and (trip_assignments[loc2] == -1):
            loc1_trip = trip_assignments[loc1]   ##CHECK LOCAITON ONE'S TRIP NUMBER 
            loc1_trip_allocation = trips[loc1_trip]['capacity_allocated']     ##LOCATION ONE'S TRIP CAPACITY
            loc1_mile_allocation = trips[loc1_trip]['miles']      ##LOCATION  ONE'S TRIP MILEAGE
            loc1_return_trip = trips[loc1_trip]['return_trip']     ## LOCATION ONE'S TRIP RETURN TRIP 
            loc1_end = trips[loc1_trip]['customers'][-1]       ## LOCATION ONE'S LAST CUSTOMER 

            capacity_required = loc1_trip_allocation + store_demand[loc2]   ## CAPACITY NECESSARY TO ADD LOCATION 2
            miles_driven = loc1_mile_allocation + d[loc1_end, loc2] - loc1_return_trip + d[loc2, test_depot]    ## NEW TOTAL MILEAGE DRIVEN

            ## IF CONSTRAINTS ARE MET 
            if capacity_required <= vehicle_capacity and miles_driven <= miles_limit:
                trips[loc1_trip]['customers'].append(loc2) ##ADDING TO TRIP
                trips[loc1_trip]['capacity_allocated'] = capacity_required  ##UPDATE CAPACITY
                trips[loc1_trip]['miles'] = miles_driven   ## UPDATE MILEAGE 
                trips[loc1_trip]['return_trip'] = d[loc2, test_depot] ## UPDATE RETURN TRIP 

                trip_assignments[loc2] = loc1_trip ## MAPPING LOCATION 2 TO TRIP 

        ## IF TRIP 2 IS MAPPED AND TRIP 1 ISNT 
        elif (trip_assignments[loc2] >= 0) and (trip_assignments[loc1] == -1):
            loc2_trip = trip_assignments[loc2] ## CHECK LOCATION 2'S TRIP NUMBER 
            loc2_trip_allocation = trips[loc2_trip]['capacity_allocated']   ##CHECK CURRENT CAPACITY LOCATION 2 TRIP
            loc2_mile_allocation = trips[loc2_trip]['miles']   ## CHECK CURRENT MILEAGE LOCATION 2 TRIP
            loc2_return_trip = trips[loc2_trip]['return_trip']  ## CHECK RETURN DISTANCE LOCATION 2 TRIP
            loc2_end = trips[loc2_trip]['customers'][-1]    ## LOCATION 2'S TRIP LAST CUSTOMER 

            capacity_required = loc2_trip_allocation + store_demand[loc1] ## COMPUTING NECESSARY CAPACITY 
            miles_driven = loc2_mile_allocation + d[loc2_end, loc1] - loc2_return_trip + d[loc1, test_depot] ## COMPUTING NECESSARY MILEAGE 

            ## IF CONSTRAINTS ARE MET 
            if capacity_required <= vehicle_capacity and miles_driven <= miles_limit:
                trips[loc2_trip]['customers'].append(loc1) ## APPEND TRIP 1
                trips[loc2_trip]['capacity_allocated'] = capacity_required ## ASSIGN NEW CAPACITY 
                trips[loc2_trip]['miles'] = miles_driven ## ASSIGN NEW MILEAGE 
                trips[loc2_trip]['return_trip'] = d[loc2, test_depot] ## ASSIGN NEW FINAL RETURN TRIP 

                trip_assignments[loc1] = loc2_trip ## MAP LOCATION 1 TO LOCATION 2 TRIP 

        ## IF BOTH LOCATIONS ARE IN A TRIP 
        else:       
            loc1_trip = trip_assignments[loc1]   ##CHECK LOCAITON ONE'S TRIP NUMBER 
            loc1_trip_allocation = trips[loc1_trip]['capacity_allocated']     ##LOCATION ONE'S TRIP CAPACITY
            loc1_mile_allocation = trips[loc1_trip]['miles']      ##LOCATION  ONE'S TRIP MILEAGE
            loc1_return_trip = trips[loc1_trip]['return_trip']     ## LOCATION ONE'S TRIP RETURN TRIP 
            loc1_end = trips[loc1_trip]['customers'][-1]       ## LOCATION ONE'S LAST CUSTOMER 

            loc2_trip = trip_assignments[loc2] ## CHECK LOCATION 2'S TRIP NUMBER 
            loc2_trip_allocation = trips[loc2_trip]['capacity_allocated']   ##CHECK CURRENT CAPACITY LOCATION 2 TRIP
            loc2_mile_allocation = trips[loc2_trip]['miles']   ## CHECK CURRENT MILEAGE LOCATION 2 TRIP
            loc2_return_trip = trips[loc2_trip]['return_trip']  ## CHECK RETURN DISTANCE LOCATION 2 TRIP
            loc2_end = trips[loc2_trip]['customers'][-1]    ## LOCATION 2'S TRIP LAST CUSTOMER 


            ## IF THEY ARE NOT IN THE SAME TRIP 
            if loc1_trip != loc2_trip:
                capacity_required = loc1_trip_allocation + loc2_trip_allocation ## CALCULATING NECESSARY CAPACITY 
                miles_driven = loc1_mile_allocation - loc1_return_trip + d[loc1_end, loc2_end] + loc2_mile_allocation + loc2_return_trip  ## CALCULATING NECESSARY MILEAGE 

                ## IF CONSTRAINTS ARE MET 
                if capacity_required <= vehicle_capacity and miles_driven <= miles_limit:
                    trips[loc1_trip]['customers'].extend(trips[loc2_trip]['customers']) ## COMBINE THE TWO TRIPS 
                    trips[loc1_trip]['capacity_allocated'] = capacity_required ## SET THE NEW CAPACITY 
                    trips[loc1_trip]['miles'] = miles_driven ## SET THE NEW MILES DRIVEN
                    trips[loc1_trip]['return_trip'] = loc2_return_trip ## SET THE NEW RETURN TRIP 

                    ## REMAPPING ALL OF THE NODES IN TRIP TWO TO TRIP ONE  
                    for customer in trips[loc2_trip]['customers']:
                        trip_assignments[customer] = loc1_trip
                    trips.pop(loc2_trip)

    ## Parameters for Calculating Total Depot Cost
    numCust = 0
    numTrucks = 0
    numMiles = 0

    ## CREATING DATA STRUCTURE FOR OUTPUTTING FULL TRIP DESCRIPTIONS                 
    trips = {idx: val for idx, val in enumerate(trips.values(), 1)}

    for trip in trips:
        numCust += len(trips[trip]['customers'])
        trips[trip]['customers'] = [test_depot] + trips[trip]['customers']
        numTrucks += 1
        numMiles += trips[trip]['miles']

    # print(trips)
    # print('Number of Trucks:', numTrucks)
    # print('Number of Miles: ', numMiles)
    # print('Number of Customers: ', numCust)

    truck_cost = 2500 * numTrucks + numMiles  
    depot_cost = 50000 + numCust*1000
    total_cost = truck_cost + depot_cost
    depot_cost_dict[test_depot] = total_cost 
    
min_cost = min(depot_cost_dict.values())
min_depot = min(depot_cost_dict, key=depot_cost_dict.get)
print(f'The best place to put a Depot which would result in the cheapest cost is at Depot #{min_depot}, costing ${round(min_cost, 2)}.')

The best place to put a Depot which would result in the cheapest cost is at Depot #801, costing $61710.01.


In [ ]:
customer_locations

In [ ]:
def plot_CW(cw_trips_dict):
    
    fig, ax = plt.subplots(1, 1, figsize=(7, 7))

    sns.scatterplot(
        x='lat',
        y='lon',
        data=coordinates,
        edgecolor='k',
        alpha=0.5,
        s=250,
    )
    
    for trip in cw_trips_dict:
        trip_customers = cw_trips_dict[trip]['customers']
        tour = solve_tsp_cb(distance_matrix.loc[trip_customers, trip_customers], verbose=False)

        for start_index, start_customer in enumerate(tour):
            if start_customer == tour[-1]:
                next_customer = tour[0]
            else:
                next_customer = tour[start_index + 1]

            xs = [coordinates.loc[start_customer, 'lat'], coordinates.loc[next_customer, 'lat']]
            ys = [coordinates.loc[start_customer, 'lon'], coordinates.loc[next_customer, 'lon']]

            ax.plot(xs, ys)

    plt.show()

In [ ]:
plot_CW(trips)

## Demand Plotting Function

In [ ]:
def plot_store_demand(selected_store_id):
    
    fig, ax = plt.subplots(1, 1, figsize=(16, 5))
    
    sns.lineplot(
        x=relevant_demand_df.index,
        y=selected_store_id,
        data=relevant_demand_df,
    )
    
    plt.show()

In [ ]:
interact(plot_store_demand, selected_store_id=relevant_stores);

## TSP Function

In [ ]:
def solve_tsp_cb(dist_df, warmstart=None, verbose=True, time_limit=None):
    """
    This function solves the travelling salesman problem for the problem defined
    by a given distance matrix. The function expects the following arguments:
    
    dist_df: a Pandas DataFrame that specifies the distance between origin-destination (OD) pairs.
    All possible values for the origin and desinatiion should be included in the index and
    columns of the DataFrame. The value at the intersection of the labels for an OD pair  
    is the distance from the origin (row label) to the destination (column label).
    
    warmstart: a list specifying a tour to use as an initial solution (optional)
    
    verbose: a flag that can be set to True or False to limit the amount of information
    from Gurobi that is written to the screen (optional)
    
    time_limit: the maximum amount of time that is allowed for Gurobi to attempt to 
    solve the problem (optional)
    
    """

    # Callback - use lazy constraints to eliminate sub-tours
    def subtourelim(model, where):
        if where == GRB.Callback.MIPSOL:
            # make a list of edges selected in the solution
            vals = model.cbGetSolution(model._vars)
            selected = tuplelist(
                (i, j) for i, j in model._vars.keys() if vals[i, j] > 0.5
            )
            # find the shortest cycle in the selected edge list
            tour = subtour(selected)
            if len(tour) < n:
                tour_sum = LinExpr()
                for tour_index, tour_stop in enumerate(tour):
                    if tour_index < len(tour) - 1:
                        tour_sum += model._vars[tour[tour_index], tour[tour_index + 1]]
                    else:
                        tour_sum += model._vars[tour[tour_index], tour[0]]
                model.cbLazy(tour_sum <= len(tour) - 1)

    # Given a list of tuples containing the tour edges, find the shortest subtour
    def subtour(edges):
        unvisited = list(dist_df.index)
        cycle = unvisited + [unvisited[0]]  # initial length has 1 more city
        while unvisited:  # true if list is non-empty
            thiscycle = []
            neighbors = unvisited
            while neighbors:
                current = neighbors[0]
                thiscycle.append(current)
                unvisited.remove(current)
                neighbors = [j for i, j in edges.select(current, "*") if j in unvisited]
            if len(cycle) > len(thiscycle):
                cycle = thiscycle
        return cycle

    # Dictionary of Euclidean distance between each pair of points
    dist = {
        (i, j): dist_df.loc[i, j]
        for i in dist_df.index
        for j in dist_df.index
        if i != j
    }

    n = len(dist_df.index)

    m = Model()
    if not verbose:
        m.params.OutputFlag = 0
    if time_limit:
        m.params.TimeLimit = time_limit
    # Create variables
    trav = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name="e")

    if warmstart:
        for warmstart_index, warmstart_stop in enumerate(warmstart):
            if warmstart_index < len(warmstart) - 1:
                trav[
                    warmstart[warmstart_index], warmstart[warmstart_index + 1]
                ].Start = 1
            else:
                trav[warmstart[warmstart_index], warmstart[0]].Start = 1
    for i in dist_df.index:
        j_sum = LinExpr()
        for j in dist_df.index:
            if j != i:
                j_sum += trav[i, j]
        m.addConstr(j_sum == 1)
    for j in dist_df.index:
        i_sum = LinExpr()
        for i in dist_df.index:
            if j != i:
                i_sum += trav[i, j]
        m.addConstr(i_sum == 1)
    # Optimize model
    m._vars = trav
    m.params.Heuristics = 0
    m.params.LazyConstraints = 1

    m.optimize(subtourelim)

    try:
        vals = m.getAttr("X", trav)
        selected = tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

        tour = subtour(selected)
        # assert len(tour) == n
        
        return [int(val) for val in tour]
    except:
        print("No solution found")

Solve TSP

In [ ]:
tour = solve_tsp_cb(distance_matrix, verbose=False)

Plot Tour

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 7))

sns.scatterplot(
    x='lon',
    y='lat',
    data=coordinates,
    edgecolor='k',
    alpha=0.5,
    s=250,
)

for start_index, start_customer in enumerate(tour):
    if start_customer == tour[-1]:
        next_customer = tour[0]
    else:
        next_customer = tour[start_index + 1]
        
    xs = [coordinates.loc[start_customer, 'lon'], coordinates.loc[next_customer, 'lon']]
    ys = [coordinates.loc[start_customer, 'lat'], coordinates.loc[next_customer, 'lat']]
    
    ax.plot(xs, ys)

plt.show()